In [5]:
# Telugu Chatbot with Live Voice Recording and Generative AI
!pip install -q transformers torch langid sounddevice numpy scipy ipywidgets
!apt-get install -qq libportaudio2

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langid import classify
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import re
from IPython.display import display, Audio, clear_output
import ipywidgets as widgets
import queue
import threading

# Initialize with CPU support
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load models
try:
    print("Loading models...")
    # Using a more capable model for Telugu
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)
    asr = pipeline("automatic-speech-recognition", model="openai/whisper-tiny", device=device)
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    raise RuntimeError("Failed to load models")

# Telugu knowledge base (now used as examples for the generative model)
telugu_examples = {
    "general": [
        {"input": "హలో", "response": "నమస్కారం! మీరు ఎలా ఉన్నారు?"},
        {"input": "ధన్యవాదాలు", "response": "స్వాగతం! మీకు ఇంకేమైనా సహాయం కావాలా?"},
        {"input": "విద్యార్థులకు సలహాలు", "response": "విద్యార్థులుగా మీరు నియమిత సమయ పట్టికను పాటించండి, రోజువారీ అభ్యాసం చేయండి మరియు సరైన ఆరోగ్యకరమైన ఆహారం తీసుకోండి."}
    ],
    "facts": {
        "తిరుపతి ఆలయం": "తిరుపతిలోని శ్రీ వేంకటేశ్వర స్వామి ఆలయం భారతదేశంలో అత్యంత ప్రసిద్ధ దేవాలయాలలో ఒకటి.",
        "చార్మినార్": "హైదరాబాద్ లోని చార్మినార్ ఒక ప్రసిద్ధ చారిత్రక స్మారకం.",
        "గోల్కొండ కోట": "గోల్కొండ కోట హైదరాబాద్ నగరం సమీపంలో ఉంది."
    }
}

def clean_telugu_text(text):
    """Clean and format Telugu text output"""
    # Remove special tokens and extra spaces
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    # Ensure proper Telugu punctuation
    text = text.replace(' .', '।').replace('.', '।')
    return text

def generate_telugu_response(user_input, chat_history=""):
    """Generate natural Telugu responses using the model"""
    # Check if it's a factual question first
    for keyword, response in telugu_examples["facts"].items():
        if keyword in user_input:
            return response
    
    # Prepare context for generative response
    context = "మీరు ఒక సహాయకుడిగా ప్రవర్తించండి. సహాయకుడిగా తెలుగులో సహజమైన, స్నేహపూర్వకమైన మరియు వివరణాత్మకమైన సమాధానాలను ఇవ్వండి.\n"
    
    # Add example conversations
    for example in telugu_examples["general"]:
        context += f"User: {example['input']}\nAssistant: {example['response']}\n"
    
    # Add current conversation history
    if chat_history:
        context += chat_history + "\n"
    
    # Add current user input
    prompt = context + f"User: {user_input}\nAssistant:"
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_beams=5,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the assistant's response
        if "Assistant:" in response:
            response = response.split("Assistant:")[-1].strip()
        
        return clean_telugu_text(response)
    except Exception as e:
        print(f"Generation error: {e}")
        return "క్షమించండి, సమస్య ఏర్పడింది. దయచేసి మళ్లీ ప్రయత్నించండి."

# Audio recording variables
audio_queue = queue.Queue()
is_recording = False
sample_rate = 16000
audio_data = []

# Create Jupyter widgets
output_area = widgets.Output()
chat_history = widgets.HTML(value="<h2 style='color:#1f618d'>తెలుగు చాట్ సహాయకుడు</h2>")
input_text = widgets.Text(placeholder='మీ ప్రశ్నను ఇక్కడ టైప్ చేయండి...', layout=widgets.Layout(width='80%'))
send_button = widgets.Button(description='పంపించు', button_style='success', icon='send')
start_button = widgets.Button(description='మాట్లాడండి 🎤', button_style='info')
stop_button = widgets.Button(description='నిలుపు ⏹️', button_style='danger', disabled=True)
clear_button = widgets.Button(description='కొత్తగా మొదలుపెట్టు', button_style='warning')

def audio_callback(indata, frames, time, status):
    """Called for each audio block when recording"""
    if is_recording:
        audio_data.append(indata.copy())

def start_recording(b):
    """Start live audio recording"""
    global is_recording, audio_data
    with output_area:
        clear_output()
        print("మాట్లాడుతూ ఉండండి... (3 సెకన్ల తర్వాత నిలుపుమని నొక్కండి)")
        is_recording = True
        audio_data = []
        start_button.disabled = True
        stop_button.disabled = False
        sd.InputStream(samplerate=sample_rate, channels=1, callback=audio_callback).start()

def stop_recording(b):
    """Stop recording and process audio"""
    global is_recording
    is_recording = False
    start_button.disabled = False
    stop_button.disabled = True
    
    with output_area:
        clear_output()
        if len(audio_data) > 0:
            # Save and play recorded audio
            recording = np.concatenate(audio_data)
            wav.write("user_audio.wav", sample_rate, recording)
            display(Audio("user_audio.wav", rate=sample_rate))
            
            # Transcribe audio
            try:
                user_input = asr("user_audio.wav")['text']
                input_text.value = user_input
                print(f"మీరు చెప్పారు: {user_input}")
                # Automatically send the voice input
                on_send(None)
            except Exception as e:
                print(f"వాయిస్ గుర్తించడంలో లోపం: {e}")
        else:
            print("ఏ ఆడియో రికార్డ్ చేయబడలేదు")

def on_send(b):
    """Handle text/voice input and generate response"""
    user_input = input_text.value.strip()
    if not user_input:
        return
        
    # Add user message to chat
    chat_history.value += f"<div style='background:#eaf2f8;padding:10px;margin:5px;border-radius:5px'><b>మీరు:</b> {user_input}</div>"
    input_text.value = ''
    
    # Generate and display response
    with output_area:
        print("ప్రతిస్పందిస్తున్నాము...")
        response = generate_telugu_response(user_input, chat_history.value)
        chat_history.value += f"<div style='background:#e8f8f5;padding:10px;margin:5px;border-radius:5px'><b>సహాయకుడు:</b> {response}</div>"
        clear_output()

def on_clear(b):
    """Clear conversation history"""
    chat_history.value = "<h2 style='color:#1f618d'>తెలుగు చాట్ సహాయకుడు</h2>"
    input_text.value = ''
    with output_area:
        clear_output()

# Set up button actions
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
send_button.on_click(on_send)
clear_button.on_click(on_clear)

# Display the interface
display(widgets.VBox([
    chat_history,
    widgets.HBox([input_text, send_button]),
    widgets.HBox([start_button, stop_button, clear_button]),
    output_area
]))


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Using device: cpu
Loading models...


Device set to use cpu


Models loaded successfully!


In [4]:
# Fully Generative Telugu Chatbot with Voice Input
!pip install -q transformers torch langid sounddevice numpy scipy ipywidgets ffmpeg-python
!apt-get install -qq libportaudio2 ffmpeg

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
from IPython.display import display, Audio, clear_output
import ipywidgets as widgets
import ffmpeg
import re

# Initialize with CPU/GPU support
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load models
print("Loading models...")
try:
    # Using a model better suited for Telugu generation
    model_name = "microsoft/DialoGPT-medium"  # Can replace with Telugu-specific model if available
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    
    # Speech recognition model
    asr = pipeline("automatic-speech-recognition", 
                  model="openai/whisper-small",  # Using small for better Telugu recognition
                  device=device,
                  chunk_length_s=30,
                  stride_length_s=5)
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    raise RuntimeError("Failed to load models")

# Audio recording variables
is_recording = False
sample_rate = 16000
audio_data = []

def save_audio_to_wav(audio_np_array, filename="user_audio.wav"):
    """Save numpy array as WAV file with proper FFmpeg conversion"""
    try:
        # First save as temporary raw file
        temp_file = "temp.raw"
        wav.write(temp_file, sample_rate, audio_np_array)
        
        # Convert to proper WAV format using FFmpeg
        (
            ffmpeg
            .input(temp_file, format='s16le', ac=1, ar=sample_rate)
            .output(filename, format='wav')
            .overwrite_output()
            .run(quiet=True)
        )
        return filename
    except Exception as e:
        print(f"Audio save error: {e}")
        return None

def transcribe_audio(audio_file):
    """Transcribe audio with error handling"""
    try:
        result = asr(audio_file)
        return result['text']
    except Exception as e:
        print(f"Transcription error: {e}")
        return None

def generate_telugu_response(user_input, chat_history=""):
    """Generate natural Telugu responses using the model"""
    # Prepare context for generative response
    context = (
        "మీరు ఒక సహాయకుడిగా ప్రవర్తించండి. తెలుగులో సహజమైన, స్నేహపూర్వకమైన మరియు వివరణాత్మకమైన సమాధానాలను ఇవ్వండి.\n"
        "ఉదాహరణ:\n"
        "User: హలో\n"
        "Assistant: నమస్కారం! మీరు ఎలా ఉన్నారు? ఏమైనా సహాయం కావాలా?\n"
        "User: తెలుగు గురించి చెప్పు\n"
        "Assistant: తెలుగు ఒక సుందరమైన భాష, ఇది భారతదేశంలో అధికంగా మాట్లాడే భాషలలో ఒకటి. దీనిని 'ఇటాలీ ఆఫ్ ది ఈస్ట్' అని కూడా పిలుస్తారు.\n"
    )
    
    # Add conversation history if available
    if chat_history:
        context += f"\n{chat_history}\n"
    
    # Add current user input
    prompt = context + f"User: {user_input}\nAssistant:"
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_beams=5,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the assistant's response
        if "Assistant:" in response:
            response = response.split("Assistant:")[-1].strip()
        
        return clean_telugu_text(response)
    except Exception as e:
        print(f"Generation error: {e}")
        return "క్షమించండి, సమస్య ఏర్పడింది. దయచేసి మళ్లీ ప్రయత్నించండి."

def clean_telugu_text(text):
    """Clean and format Telugu text output"""
    text = re.sub(r'<[^>]+>', '', text)  # Remove any HTML tags
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = text.replace(' .', '।').replace('.', '।')  # Proper Telugu punctuation
    return text

# Create Jupyter widgets
output_area = widgets.Output()
chat_history = widgets.HTML(value="<h2 style='color:#1f618d'>తెలుగు జనరేటివ్ చాట్‌బాట్</h2>")
input_text = widgets.Text(placeholder='మీ ప్రశ్నను ఇక్కడ టైప్ చేయండి...', layout=widgets.Layout(width='80%'))
send_button = widgets.Button(description='పంపించు', button_style='success', icon='send')
start_button = widgets.Button(description='మాట్లాడండి 🎤', button_style='info')
stop_button = widgets.Button(description='నిలుపు ⏹️', button_style='danger', disabled=True)
clear_button = widgets.Button(description='కొత్తగా మొదలుపెట్టు', button_style='warning')

def audio_callback(indata, frames, time, status):
    """Called for each audio block when recording"""
    if is_recording:
        audio_data.append(indata.copy())

def start_recording(b):
    """Start live audio recording"""
    global is_recording, audio_data
    with output_area:
        clear_output()
        print("మాట్లాడుతూ ఉండండి... (3-5 సెకన్ల తర్వాత నిలుపుమని నొక్కండి)")
        is_recording = True
        audio_data = []
        start_button.disabled = True
        stop_button.disabled = False
        sd.InputStream(samplerate=sample_rate, channels=1, callback=audio_callback).start()

def stop_recording(b):
    """Stop recording and process audio"""
    global is_recording
    is_recording = False
    start_button.disabled = False
    stop_button.disabled = True
    
    with output_area:
        clear_output()
        if len(audio_data) > 0:
            try:
                # Convert audio data to numpy array
                recording = np.concatenate(audio_data)
                
                # Save audio with proper format conversion
                audio_file = save_audio_to_wav(recording)
                if not audio_file:
                    raise ValueError("Audio file could not be saved")
                
                # Play back the recording
                display(Audio(audio_file, rate=sample_rate))
                
                # Transcribe audio
                user_input = transcribe_audio(audio_file)
                if user_input:
                    input_text.value = user_input
                    print(f"మీరు చెప్పారు: {user_input}")
                    # Automatically send the transcribed text
                    on_send(None)
                else:
                    print("వాయిస్ గుర్తించడంలో లోపం. దయచేసి మళ్లీ ప్రయత్నించండి.")
            except Exception as e:
                print(f"ఆడియో ప్రాసెసింగ్ లోపం: {e}")
        else:
            print("ఏ ఆడియో రికార్డ్ చేయబడలేదు")

def on_send(b):
    """Handle text/voice input and generate response"""
    user_input = input_text.value.strip()
    if not user_input:
        return
        
    # Add user message to chat
    chat_history.value += f"<div style='background:#eaf2f8;padding:10px;margin:5px;border-radius:5px'><b>మీరు:</b> {user_input}</div>"
    input_text.value = ''
    
    # Generate and display response
    with output_area:
        print("ప్రతిస్పందిస్తున్నాము...")
        response = generate_telugu_response(user_input, chat_history.value)
        chat_history.value += f"<div style='background:#e8f8f5;padding:10px;margin:5px;border-radius:5px'><b>సహాయకుడు:</b> {response}</div>"
        clear_output()

def on_clear(b):
    """Clear conversation history"""
    chat_history.value = "<h2 style='color:#1f618d'>తెలుగు జనరేటివ్ చాట్‌బాట్</h2>"
    input_text.value = ''
    with output_area:
        clear_output()

# Set up button actions
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)
send_button.on_click(on_send)
clear_button.on_click(on_clear)

# Display the interface
display(widgets.VBox([
    chat_history,
    widgets.HBox([input_text, send_button]),
    widgets.HBox([start_button, stop_button, clear_button]),
    output_area
]))


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Using device: cpu
Loading models...


Device set to use cpu


Models loaded successfully!


In [3]:
# Telugu Text Chatbot with Knowledge Base
!pip install -q transformers torch ipywidgets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import ipywidgets as widgets
import re

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load models
print("Loading models...")
try:
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    raise

# Telugu Knowledge Base for Indian Places
knowledge_base = {
    # Temples
    "తిరుపతి": {
        "description": ("తిరుపతిలోని శ్రీ వేంకటేశ్వర స్వామి ఆలయం ప్రసిద్ధ హిందూ దేవాలయం. "
                      "ఇది ఆంధ్రప్రదేశ్ లోని తిరుమల పర్వతాల మీద ఉంది. "
                      "ప్రతి సంవత్సరం లక్షలాది భక్తులు ఇక్కడకు దర్శనం కోసం వస్తారు."),
        "tags": ["తిరుమల", "వేంకటేశ్వర", "దేవాలయం"]
    },
    "వారణాసి": {
        "description": ("వారణాసి (కాశీ) గంగా నది ఒడ్డున ఉన్న పుణ్యనగరం. "
                      "ఇది ప్రపంచంలోనే అత్యంత పురాతన నగరాలలో ఒకటి. "
                      "కాశీ విశ్వనాథ్ దేవస్థానం ఇక్కడి ప్రధాన ఆకర్షణ."),
        "tags": ["కాశీ", "గంగా", "విశ్వనాథ్"]
    },
    
    # Historical Sites
    "తాజ్ మహల్": {
        "description": ("తాజ్ మహల్ ఉత్తరప్రదేశ్ లోని ఆగ్రాలో ఉన్న పాలరాయి మక్బరా. "
                      "మొఘల్ చక్రవర్తి షాజహాన్ తన భార్య ముమ్తాజ్ మహల్ కోసం నిర్మించారు. "
                      "ఇది ప్రపంచ ఏడు అద్భుతాలలో ఒకటిగా పరిగణించబడుతుంది."),
        "tags": ["ఆగ్రా", "మొఘల్", "ప్రేమ స్మారకం"]
    },
    "ఢిల్లీ": {
        "description": ("భారతదేశ రాజధాని ఢిల్లీ చారిత్రక ప్రాముఖ్యత కలిగిన నగరం. "
                      "ఇక్కడ ఎర్రకోట, కుతుబ్ మినార్, లోటస్ టెంపుల్ వంటి ప్రసిద్ధ స్మారకాలు ఉన్నాయి. "
                      "ఇది దేశం యొక్క రాజకీయ, సాంస్కృతిక కేంద్రం."),
        "tags": ["రాజధాని", "ఎర్రకోట", "కుతుబ్ మినార్"]
    },
    
    # Natural Wonders
    "కేరళ": {
        "description": ("కేరళను 'దేవుని స్వంత నాడు' అని పిలుస్తారు. "
                      "బ్యాక్ వాటర్స్, కొక్కోనట్ బీచ్‌లు, అడవులు, ఆయుర్వేదం ఇక్కడి ప్రత్యేకతలు. "
                      "మున్నార్, అల్లెప్పీ, కొచ్చి ప్రముఖ పర్యాటక స్థలాలు."),
        "tags": ["బ్యాక్ వాటర్స్", "మున్నార్", "ఆయుర్వేదం"]
    },
    "లడఖ్": {
        "description": ("లడఖ్ హిమాలయాలలో ఉన్న ఒక శీతల మరుభూమి. "
                      "పంగాంగ్ సరోవర్, నుబ్రా వ్యాలీ, బుద్ధ మఠాలు ఇక్కడి ప్రధాన ఆకర్షణలు. "
                      "ఇది ట్రెక్కింగ్ మరియు సాహస యాత్రలకు ప్రసిద్ధి చెందింది."),
        "tags": ["హిమాలయాలు", "లేహ్", "ట్రెక్కింగ్"]
    },
    
    # Cities
    "ముంబై": {
        "description": ("ముంబై భారతదేశ ఆర్థిక రాజధాని. "
                      "గేట్వే ఆఫ్ ఇండియా, మరీన్ డ్రైవ్, బాలీవుడ్ ఇక్కడి ప్రధాన ఆకర్షణలు. "
                      "ఇది భారతదేశంలోని అత్యధిక జనసాంద్రత కలిగిన నగరం."),
        "tags": ["బాలీవుడ్", "గేట్వే", "ఆర్థిక రాజధాని"]
    },
    "బెంగళూరు": {
        "description": ("బెంగళూరును 'సిలికాన్ వ్యాలీ ఆఫ్ ఇండియా' అని పిలుస్తారు. "
                      "ఇది భారతదేశ ఐటీ రాజధాని. "
                      "లాల్ బాగ్, బెంగళూరు ప్యాలెస్, ఐఎస్‌కెఆన్ ఇక్కడి ప్రధాన ఆకర్షణలు."),
        "tags": ["ఐటీ", "సిలికాన్ వ్యాలీ", "గార్డెన్ సిటీ"]
    },
    
    # Pilgrimage
    "అమృతసర్": {
        "description": ("అమృతసర్ లోని స్వర్ణదేవాలయం (గోల్డెన్ టెంపుల్) సిక్కు మతానికి ప్రధాన పవిత్ర స్థలం. "
                      "ఇక్కడి లంగర్ (సామూహిక భోజనం) ప్రసిద్ధి చెందింది. "
                      "ఇది పంజాబ్ రాష్ట్రంలో ఉంది."),
        "tags": ["గోల్డెన్ టెంపుల్", "సిక్కులు", "లంగర్"]
    },
    "రామేశ్వరం": {
        "description": ("రామేశ్వరం తమిళనాడులో ఉన్న పవిత్ర హిందూ తీర్థయాత్రా కేంద్రం. "
                      "రామనాథస్వామి దేవస్థానం ఇక్కడి ప్రధాన ఆకర్షణ. "
                      "హిందూ పురాణాల ప్రకారం శ్రీరాముడు ఇక్కడే రావణుని వద్దకు సేతువు నిర్మించాడు."),
        "tags": ["సేతు", "రామనాథస్వామి", "తీర్థయాత్ర"]
    },
    
    # Added more places
    "హైదరాబాద్": {
        "description": ("హైదరాబాద్ తెలంగాణ రాజధాని. "
                      "చార్మినార్, గోల్కొండ కోట, రామోజీ ఫిల్మ్ సిటీ ఇక్కడి ప్రధాన ఆకర్షణలు. "
                      "ఇది భారతదేశ ఫార్మా మరియు ఐటీ హబ్."),
        "tags": ["చార్మినార్", "బిర్యానీ", "టెక్ సిటీ"]
    },
    "జైపూర్": {
        "description": ("జైపూర్ను 'పింక్ సిటీ' అని పిలుస్తారు. "
                      "హవా మహల్, అంబర్ కోట, జంతర్ మంతర్ ఇక్కడి ప్రధాన ఆకర్షణలు. "
                      "ఇది రాజస్థాన్ రాజధాని మరియు గోల్డెన్ ట్రయాంగిల్ లో భాగం."),
        "tags": ["పింక్ సిటీ", "హవా మహల్", "రాజస్థాన్"]
    }
}

def generate_telugu_response(user_input):
    """Generate Telugu responses using knowledge base or generative model"""
    # Check knowledge base first
    for place, info in knowledge_base.items():
        if place.lower() in user_input.lower() or any(tag in user_input for tag in info["tags"]):
            return info["description"]
    
    # Generate response if not in knowledge base
    try:
        prompt = f"తెలుగులో సమాధానం ఇవ్వండి: {user_input}"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return clean_telugu_text(response.split(":")[-1].strip())
    except Exception as e:
        print(f"Error: {e}")
        return "క్షమించండి, సమస్య ఏర్పడింది. దయచేసి మళ్లీ ప్రయత్నించండి."

def clean_telugu_text(text):
    """Clean and format Telugu text output"""
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace(' .', '।').replace('.', '।')
    if not text.endswith(('ం', 'ు', 'ి', 'ీ', 'ా', 'ో', 'ూ', '్', '?')):
        text += '।'
    return text

# Create widgets
output_area = widgets.Output()
chat_history = widgets.HTML(value="<h2 style='color:#1f618d'>తెలుగు చాట్‌బాట్</h2>")
input_text = widgets.Text(placeholder='మీ ప్రశ్నను ఇక్కడ టైప్ చేయండి...', layout=widgets.Layout(width='80%'))
send_button = widgets.Button(description='పంపించు', button_style='success')
clear_button = widgets.Button(description='క్లియర్', button_style='warning')

def on_send(b):
    user_input = input_text.value.strip()
    if not user_input:
        with output_area:
            print("దయచేసి ఏదో టైప్ చేయండి")
        return
    
    # Add user message
    chat_history.value += f"<div style='margin:10px;padding:10px;background:#e6f3ff;border-radius:5px'><b>మీరు:</b> {user_input}</div>"
    input_text.value = ''
    
    with output_area:
        print("ప్రతిస్పందిస్తున్నాము...")
        try:
            response = generate_telugu_response(user_input)
            chat_history.value += f"<div style='margin:10px;padding:10px;background:#e6ffe6;border-radius:5px'><b>సహాయకుడు:</b> {response}</div>"
        except Exception as e:
            chat_history.value += f"<div style='margin:10px;padding:10px;background:#ffebee;border-radius:5px'><b>లోపం:</b> {str(e)}</div>"
        finally:
            clear_output()

def on_clear(b):
    chat_history.value = "<h2 style='color:#1f618d'>తెలుగు చాట్‌బాట్</h2>"
    with output_area:
        clear_output()

# Connect buttons
send_button.on_click(on_send)
clear_button.on_click(on_clear)

# Display interface
display(widgets.VBox([
    chat_history,
    widgets.HBox([input_text, send_button]),
    clear_button,
    output_area
]))

print("చాట్‌బాట్ సిద్ధంగా ఉంది! ప్రశ్నలు అడగండి.")


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Using device: cpu
Loading models...
Models loaded successfully!


చాట్‌బాట్ సిద్ధంగా ఉంది! ప్రశ్నలు అడగండి.


In [1]:
import os
import wave
import pyaudio
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from gtts import gTTS
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import io
import warnings
from IPython.display import display, clear_output, Audio
import ipywidgets as widgets

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Define the knowledge base FIRST - before any functions that use it
knowledge_base = {
    "tirupati": {
        "description": "తిరుపతిలోని శ్రీ వేంకటేశ్వర స్వామి ఆలయం ప్రసిద్ధ హిందూ దేవాలయం. ఇది ఆంధ్రప్రదేశ్ లోని తిరుమల పర్వతాల మీద ఉంది. ఇక్కడ ప్రతి సంవత్సరం లక్షలాది భక్తులు దర్శనం కోసం వస్తారు.",
        "tags": ["తిరుపతి", "వేంకటేశ్వర", "తిరుమల", "ఆంధ్రప్రదేశ్"]
    },
    "hyderabad": {
        "description": "హైదరాబాద్ లో చార్మినార్, గోల్కొండ కోట, రామోజీ ఫిల్మ్ సిటీ ప్రధాన ఆకర్షణలు. హైదరాబాద్ తెలంగాణ రాష్ట్ర రాజధాని.",
        "tags": ["హైదరాబాద్", "చార్మినార్", "గోల్కొండ", "రామోజీ ఫిల్మ్ సిటీ", "తెలంగాణ"]
    },
    "vijayawada": {
        "description": "విజయవాడ ఆంధ్రప్రదేశ్ లోని ఒక ప్రధాన నగరం. ఇది కృష్ణా నది ఒడ్డున ఉంది మరియు వ్యాపార, వాణిజ్య కేంద్రంగా ప్రసిద్ధి చెందింది. కనకదుర్గ మల్లేశ్వర స్వామి ఆలయం ఇక్కడి ప్రధాన ఆకర్షణ.",
        "tags": ["విజయవాడ", "కృష్ణా", "కనకదుర్గ", "ఆంధ్రప్రదేశ్"]
    }
}

# Load models
print("Loading models...")
try:
    # Embedding model
    embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)
    
    # Speech recognition - Using Whisper medium model for better Telugu recognition
    asr = pipeline("automatic-speech-recognition", 
                  model="openai/whisper-medium",
                  device=0 if torch.cuda.is_available() else -1)
    
    # Text generation
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").to(device)
    
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    raise

# Pre-compute embeddings for all knowledge base items
for key in knowledge_base:
    knowledge_base[key]["embedding"] = embedder.encode(knowledge_base[key]["description"])

def clean_telugu_text(text):
    """Clean and normalize Telugu text"""
    import re
    text = re.sub(r'[^\u0C00-\u0C7F\s.,!?]', '', text)
    text = ' '.join(text.split())
    return text.strip()

def retrieve_knowledge(query, top_k=2):
    """Retrieve relevant knowledge"""
    cleaned_query = clean_telugu_text(query)
    
    # First try exact matches with knowledge base keys
    if cleaned_query.lower() in [k.lower() for k in knowledge_base.keys()]:
        return [knowledge_base[cleaned_query.lower()]["description"]]
    
    # Then try tag matches
    matched_items = []
    for key in knowledge_base:
        item = knowledge_base[key]
        if any(tag.lower() in cleaned_query.lower() for tag in item["tags"]):
            matched_items.append(item["description"])
    
    if matched_items:
        return matched_items[:top_k]
    
    # Semantic search fallback
    query_embedding = embedder.encode(cleaned_query)
    similarities = []
    
    for key in knowledge_base:
        item = knowledge_base[key]
        sim = cosine_similarity([query_embedding], [item["embedding"]])[0][0]
        similarities.append((sim, item["description"]))
    
    similarities.sort(reverse=True, key=lambda x: x[0])
    return [text for (sim, text) in similarities[:top_k] if sim > 0.5]

def generate_telugu_response(user_input):
    """Generate context-aware Telugu response."""
    cleaned_input = clean_telugu_text(user_input)
    context = retrieve_knowledge(cleaned_input)
    
    if context:
        if len(context) == 1:
            return context[0]
        prompt = f"సందర్భం: {' '.join(context)}\n\nప్రశ్న: {cleaned_input}\nసమాధానం:"
    else:
        prompt = f"తెలుగులో సమాధానం ఇవ్వండి: {cleaned_input}"
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return clean_telugu_text(response.split("సమాధానం:")[-1].strip())
    except Exception as e:
        print(f"Generation error: {e}")
        return "క్షమించండి, సమాధానం ఇవ్వలేకపోయాను. దయచేసి మరోసారి ప్రయత్నించండి."

def text_to_speech(text, lang='te'):
    """Convert text to speech"""
    cleaned_text = clean_telugu_text(text)
    if not cleaned_text or cleaned_text.strip() == "":
        return None
        
    try:
        tts = gTTS(text=cleaned_text, lang=lang, slow=False)
        audio_file = io.BytesIO()
        tts.write_to_fp(audio_file)
        audio_file.seek(0)
        return audio_file
    except Exception as e:
        print(f"TTS Error: {e}")
        return None

def record_audio(duration=5, sample_rate=16000):
    """Record audio with better error handling"""
    try:
        chunk = 1024
        format = pyaudio.paInt16
        channels = 1
        p = pyaudio.PyAudio()

        # Check available devices
        info = p.get_default_input_device_info()
        if info['maxInputChannels'] == 0:
            raise ValueError("మైక్రోఫోన్ కనుగొనబడలేదు. దయచేసి మీ మైక్రోఫోన్ కనెక్ట్ చేయండి.")

        stream = p.open(format=format,
                        channels=channels,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=chunk,
                        input_device_index=info['index'])
        
        print("రికార్డింగ్ ప్రారంభమైంది... దయచేసి ఇప్పుడు మాట్లాడండి")
        frames = []
        
        for _ in range(0, int(sample_rate / chunk * duration)):
            data = stream.read(chunk, exception_on_overflow=False)
            frames.append(data)

        stream.stop_stream()
        stream.close()
        p.terminate()

        # Check if audio was recorded
        if len(frames) == 0:
            raise ValueError("ఆడియో రికార్డ్ కాలేదు")

        wav_buffer = io.BytesIO()
        with wave.open(wav_buffer, 'wb') as wf:
            wf.setnchannels(channels)
            wf.setsampwidth(p.get_sample_size(format))
            wf.setframerate(sample_rate)
            wf.writeframes(b''.join(frames))
        
        wav_buffer.seek(0)
        return wav_buffer
    except Exception as e:
        print(f"Recording error: {e}")
        return None

# Create widgets
output_area = widgets.Output()
chat_history = widgets.HTML(
    value="<div style='font-family:Arial;'>"
          "<h2 style='color:#1f618d'>తెలుగు వాయిస్ చాట్‌బాట్</h2>"
          "<p>దయచేసి మైక్రోఫోన్‌ను అనుమతించండి మరియు స్పష్టంగా మాట్లాడండి</p>"
          "</div>"
)

record_button = widgets.Button(
    description="మాట్లాడండి (5 సెకన్లు)", 
    button_style='success',
    icon='microphone',
    layout=widgets.Layout(width='200px', height='50px')
)

clear_button = widgets.Button(
    description="క్లియర్ చేయండి", 
    button_style='warning',
    icon='trash',
    layout=widgets.Layout(width='150px', height='50px')
)

button_box = widgets.HBox([record_button, clear_button], 
                         layout=widgets.Layout(justify_content='center'))

main_container = widgets.VBox([
    chat_history,
    button_box,
    output_area
], layout=widgets.Layout(
    width='80%', 
    margin='0 auto',
    padding='20px',
    border='2px solid #1f618d',
    border_radius='10px',
    box_shadow='0 4px 8px 0 rgba(0,0,0,0.2)'
))

def on_record(b):
    with output_area:
        clear_output()
        try:
            print("దయచేసి 5 సెకన్ల పాటు స్పష్టంగా మాట్లాడండి...")
            audio_buffer = record_audio()
            
            if audio_buffer is None:
                raise ValueError("మైక్రోఫోన్ పనిచేయడం లేదు లేదా ఆడియో రికార్డ్ కాలేదు.")
                
            print("మీ వాయిస్ రికార్డింగ్ ప్లే అవుతుంది...")
            display(Audio(audio_buffer.read(), autoplay=True))
            audio_buffer.seek(0)
            
            print("మీ మాటలను టెక్స్ట్‌గా మారుస్తున్నాము...")
            audio_data = np.frombuffer(audio_buffer.read(), dtype=np.int16)
            
            # Convert and normalize audio data
            audio_data = audio_data.astype(np.float32) / 32768.0
            
            # Use ASR with Telugu language setting
            transcript = asr(audio_data, generate_kwargs={"language": "telugu"})["text"]
            
            cleaned_transcript = clean_telugu_text(transcript)
            print(f"గుర్తించిన టెక్స్ట్: {cleaned_transcript}")
            
            if not cleaned_transcript.strip():
                raise ValueError("మాటలు స్పష్టంగా గుర్తించబడలేదు. దయచేసి మరోసారి ప్రయత్నించండి.")
                
            print(f"మీ ప్రశ్న: {cleaned_transcript}")
            
            response = generate_telugu_response(cleaned_transcript)
            print(f"సమాధానం: {response}")
            
            audio_response = text_to_speech(response)
            if audio_response:
                print("సమాధానం ఆడియో ప్లే అవుతుంది...")
                display(Audio(audio_response.read(), autoplay=True))
            
            chat_history.value += f"""
            <div style='font-family:Arial; margin:10px 0; padding:10px; border-radius:5px; background-color:#f8f9f9;'>
                <div style='color:#1a5276; font-weight:bold;'>మీరు:</div>
                <div style='margin-left:15px;'>{cleaned_transcript}</div>
                <div style='color:#27ae60; font-weight:bold; margin-top:5px;'>చాట్‌బాట్:</div>
                <div style='margin-left:15px;'>{response}</div>
            </div>
            """
            
        except Exception as e:
            error_msg = f"ప్రాసెస్ చేయడంలో లోపం: {str(e)}"
            print(error_msg)
            chat_history.value += f"""
            <div style='font-family:Arial; color:#c0392b; margin:10px 0; padding:10px; background-color:#fadbd8; border-radius:5px;'>
                {error_msg}
            </div>
            """

def on_clear(b):
    chat_history.value = "<div style='font-family:Arial;'>" \
                        "<h2 style='color:#1f618d'>తెలుగు వాయిస్ చాట్‌బాట్</h2>" \
                        "<p>దయచేసి మైక్రోఫోన్‌ను అనుమతించండి మరియు స్పష్టంగా మాట్లాడండి</p>" \
                        "</div>"
    with output_area:
        clear_output()

# Connect buttons
record_button.on_click(on_record)
clear_button.on_click(on_clear)

# Display interface
display(main_container)


Using device: cpu
Loading models...


Device set to use cpu


Models loaded successfully!


In [2]:
# Image Captioning Bot with Deep Translator for Telugu
#!pip install -q transformers torch pillow ipywidgets deep-translator
#!apt-get install -qq libgl1-mesa-glx

import torch
from transformers import pipeline, BlipForConditionalGeneration, BlipProcessor
from deep_translator import GoogleTranslator
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
import io
import base64
from io import BytesIO

# Initialize device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize translator
translator = GoogleTranslator(source='en', target='te')

# Load models
print("Loading models...")
try:
    # Image captioning model
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
    print("Models loaded successfully!")
except Exception as e:
    print(f"Error loading models: {e}")
    raise

def generate_caption(image):
    """Generate English caption for an image and translate to Telugu"""
    try:
        # Convert to RGB if needed
        if image.mode != "RGB":
            image = image.convert("RGB")
            
        # Generate English caption
        inputs = processor(image, return_tensors="pt").to(device)
        outputs = caption_model.generate(**inputs, max_new_tokens=50)
        english_caption = processor.decode(outputs[0], skip_special_tokens=True)
        
        # Translate to Telugu using Deep Translator
        telugu_caption = translator.translate(english_caption)
        
        return english_caption, telugu_caption
        
    except Exception as e:
        print(f"Captioning error: {e}")
        return "Could not generate caption", "వివరణను రూపొందించలేకపోయాము"

def image_to_base64(image):
    """Convert image to base64 for display"""
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# Create widgets
output_area = widgets.Output()
chat_history = widgets.HTML(value="<h2 style='color:#1f618d'>చిత్ర వివరణ జనరేటర్</h2>")
upload_button = widgets.FileUpload(
    description="చిత్రం అప్లోడ్ చేయండి",
    accept='image/*',
    multiple=False
)
generate_button = widgets.Button(description="వివరణను రూపొందించు", button_style="success")
clear_button = widgets.Button(description="క్లియర్", button_style="warning")

def on_generate(b):
    if not upload_button.value:
        with output_area:
            print("దయచేసి ముందు ఒక చిత్రం అప్లోడ్ చేయండి")
        return
    
    with output_area:
        clear_output()
        print("చిత్రాన్ని ప్రాసెస్ చేస్తున్నాము...")
        
        # Get uploaded image
        uploaded = upload_button.value[0]
        image_bytes = uploaded['content']
        image = Image.open(io.BytesIO(image_bytes))
        
        # Display image
        display(image)
        
        # Generate and display captions
        english_caption, telugu_caption = generate_caption(image)
        chat_history.value += f"""
        <div style='margin:10px; padding:10px; border:1px solid #eee; border-radius:5px'>
            <img src='data:image/png;base64,{image_to_base64(image)}' width='300'/>
            <p><b>English:</b> {english_caption}</p>
            <p><b>తెలుగు:</b> {telugu_caption}</p>
        </div>
        """
        print(f"వివరణ రూపొందించబడింది!")

def on_clear(b):
    chat_history.value = "<h2 style='color:#1f618d'>చిత్ర వివరణ జనరేటర్</h2>"
    upload_button.value = []
    with output_area:
        clear_output()

# Connect buttons
generate_button.on_click(on_generate)
clear_button.on_click(on_clear)

# Display interface
display(widgets.VBox([
    chat_history,
    widgets.HBox([upload_button, generate_button, clear_button]),
    output_area
]))

print("చిత్రాలకు వివరణలు రూపొందించడానికి సిద్ధంగా ఉంది! చిత్రం అప్లోడ్ చేసి బటన్ నొక్కండి")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Using device: cpu
Loading models...


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Models loaded successfully!


చిత్రాలకు వివరణలు రూపొందించడానికి సిద్ధంగా ఉంది! చిత్రం అప్లోడ్ చేసి బటన్ నొక్కండి
